# 클래스(class)와 캡슐화(encapsulation)

클래스(레코드)를 설계할 때 되도록이면
Encapsulation 원리/원칙, 요 성질이 잘 지켜져야 된다

객체의 상태는 그 객체의 메소드를 통해서만 변경 가능해야 한다는 이 원칙을 완전히 지키려면
- 객체 상태 공유를 아예 안하거나
- 변경이 불가능한 것만 공유하거나
- 직접 공유하지 않고 변경 불가능하게 포장해서 간접적으로 공유
  - 간접적으로 공유해 주는 객체(즉, 변경 가능한 데이터의 주인)는 캡슐화 원리 지켜짐
  - 공유를 받는 쪽에서 공유받은 참조를 객체 상태의 일부로 저장하는 경우에는, 공유받은 쪽은 캡슐화 원리가 깨짐

Let's say i want to create a library of books,
so we need a record Book and a record Library that stores the books has a list


In [1]:
record Book(String title, String author) { }
record Library(List<Book> books) { }


and use it that way


In [2]:
var book = new Book("DaVinci Code", "Dan Brown");
var books = new ArrayList<Book>();
books.add(book);
var library = new Library(books);
System.out.println(library);


Library[books=[Book[title=DaVinci Code, author=Dan Brown]]]


The problem with a Library declared like this in that the library is not really
in control of the books inside itself, one can write


In [3]:
books.add(new Book("Effective Java", "Joshua Bloch"));
System.out.println(library);


Library[books=[Book[title=DaVinci Code, author=Dan Brown], Book[title=Effective Java, author=Joshua Bloch]]]


The result is surprising, you can add books in the library without calling
a method of the library which make the code hard to debug because changing
an object has an effect to another object.


In [4]:
// 맨 처음 초기화를 위해 사용한 books 변수가 아니라도
// 또 다른 참조로 레코드 내부 정보를 추가 공유도 가능
var books2 = library.books();
books2.add(new Book("홍길동전","허균"));

true

In [5]:
library

Library[books=[Book[title=DaVinci Code, author=Dan Brown], Book[title=Effective Java, author=Joshua Bloch], Book[title=홍길동전, author=허균]]]

## Encapsulation principle
In a pure functional language, the language doesn't allow you to
do side effect. In an OO language, if you want to survive, the idea is
to limit the functions that can do side effects to the instance methods.


This idea is named the encapsulation principle and is sum up by this sentence
> The only way to change the value of an object is to use one of the methods of this object.


In Java, the way to ensure the encapsulation principle is to do information hiding,
i.e. to separate the __public__ API part (what the user code can use) from the __private__
implementation part (how the class is implemented).


This separation is done by using a special syntax named __class__ that allows
to precisely control of the visibility of its members.


## Class
A class defines
- private fields that is like a record component but not visible by the user code
- a public constructor (Library), that guarantee that any objects will be correctly initialized
- public and private instance and static methods


### Unmodifiable class


In [6]:
class Library {
  private final List<Book> books;
  public Library(List<Book> books) {
    this.books = List.copyOf(books);
  }
  public String toString() {
    return "Library " + books.toString();
  }
}
var library = new Library(books);
System.out.println(library);


Library [Book[title=DaVinci Code, author=Dan Brown], Book[title=Effective Java, author=Joshua Bloch], Book[title=홍길동전, author=허균]]


Now changing the list of books has no effect on the library
because the field `books` and the argument of the constructor `books` are different references


In [7]:
books.remove(new Book("DaVinci Code", "Dan Brown"));
books.remove(new Book("홍길동전", "허균"));
System.out.println(books);
System.out.println(library);


[Book[title=Effective Java, author=Joshua Bloch]]
Library [Book[title=DaVinci Code, author=Dan Brown], Book[title=Effective Java, author=Joshua Bloch], Book[title=홍길동전, author=허균]]


You can notice that the constructor has no return type, it's because it's always void.


The field 'books' is declared final which means must be initialized
in the constructor (and not changed afterward) so we are sure that in toString(),
the field 'books' has been initialized.


Unlike a record, the method equals()/hashCode() and toString() are not provided and has
to be hand written. We will see how to implement them later.


### Modifiable class
The code above is an unmodifiable implementation of Library.
We can also write a mutable version with the caveat that using it
as element of a list or a map is not recommended.


In [8]:
class ModifiableLibrary {
  private final ArrayList<Book> books;
  public ModifiableLibrary() {
    books = new ArrayList<>();
  }
  public void add(Book book) {
    Objects.requireNonNull(book);
    books.add(book);
  }
  public String toString() {
    return "ModifiableLibrary " + books.toString();
  }
}
var library = new ModifiableLibrary();
library.add(new Book("DaVinci Code", "Dan Brown"));
System.out.println(library);
library.add(new Book("Effective Java", "Joshua Bloch"));
System.out.println(library);


ModifiableLibrary [Book[title=DaVinci Code, author=Dan Brown]]
ModifiableLibrary [Book[title=DaVinci Code, author=Dan Brown], Book[title=Effective Java, author=Joshua Bloch]]


### Modifiable class and accessors
An error sometime seen is to add a method to get the content of the library
and forget that it may expose the private list of books


In [9]:
class ModifiableLibrary {
  private final ArrayList<Book> books;
  public ModifiableLibrary() {
    books = new ArrayList<>();
  }
  public void add(Book book) {
    Objects.requireNonNull(book);
    books.add(book);
  }
  public List<Book> getBooks() {
    return books;
  }
  public String toString() {
    return "ModifiableLibrary " + books.toString();
  }
}


The following code breaks the encapsulation because you can 
modify the library without calling a method of the Library
(`add()` is called on the List<Book> not on the Library)


In [10]:
var library = new ModifiableLibrary();
var books = library.getBooks();
books.add(new Book("DaVinci Code", "Dan Brown"));


true

One solution is to return a copy, or better a non modifiable view
of the internal list of books


In [11]:
class ModifiableLibrary {
    private final ArrayList<Book> books;
    public ModifiableLibrary() { books = new ArrayList<>(); }
    public void add(Book book) { books.add(book); }
    public List<Book> getBooks() {
        // // 매번 복사본을 만드는 것도 가능하지만 다소 비효율적
        // return List.copyOf(books); 
        
        // set, add 등의 변경 연산을 시도하면 에러가 나도록 포장한 view를 리턴
        return Collections.unmodifiableList(books); 
    }
    public String toString() { return "ModifiableLibrary " + books.toString(); }
}

In [12]:
var library = new ModifiableLibrary();
var books = library.getBooks();
System.out.println(books);

[]


In [13]:
books.add(new Book("DaVinci Code", "Dan Brown"));

EvalException: 

The best solution being to not have a method `getBook()` at all,
the less code you write the less bug you have.
So please don't write getters and setters unless you really need them.


## Record constructor
Records also provides ways to customize the code to respect the
encapsulation principle
Here, we only need to change the canonical constructor 


In [14]:
record Library(List<Book> books) {
  public Library(List<Book> books) {
    this.books = List.copyOf(books);
  }
}


To summarize, a class is a general mechanism to describe how things
are implemented and make a separation between what is publicly visible
and what is privately implemented to make the code working.
A record is a special case when there is no separation, everything is public.


----

용어 정리

클래스(class) 안에 작성되는 내용들
 - **필드**(field): 멤버 변수(member variable)라고도 부름
   - 인스턴스 필드(instance field) = 인스턴스 변수(instance variable)
   - 클래스 필드(class field) = 클래스 변수(class variable)
 - **메소드**/메서드(method): 최신 외래어 표기법에 따르면 "메서드"가 더 맞다고 함. 다른 객체지향 언어에서는 멤버 함수(member function)라는 용어를 쓰기도 함.
   - 클래스 메소드(class method): 정적 메소드(static method)라고 부르기도 함
   - 인스턴스 메소드(instance method)
 - **생성자**(constructor)
 - 기타 다른 내용들: nested class, enum, ...


자바에서 값을 담는 변수(variable)의 종류
 - 클래스/레코드 안에서 선언하는 필드
 - 메소드의 파라메터
 - 메소드 안에서 선언하는 지역 변수(local variable)

참고자료
- https://docs.oracle.com/javase/tutorial/java/javaOO/variables.html
